In [ ]:
# continuing with joins, using some pre-loaded tables in postgres to save time

In [1]:
import sys
import pandas as pd
from pandas.io import sql
from pandas.io.sql import read_sql
from pandas.io.sql import to_sql

#sqlalchemy
import sqlalchemy
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, ForeignKey

# for postgres
import psycopg2

import warnings
warnings.filterwarnings("ignore")

print('OK')

OK


/Users/tbroderick/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# open a new connection to pyanalysis
engine = create_engine('postgresql://postgres@localhost:5432/pyanalysis')
print("OK")

OK


In [3]:
# selecting specfic rows, joining three tables
# and simplifying column names all at the same time
    
sql = """
    SELECT lt.id, lt.left_school, en.enrollment, gr.grades
    FROM schools_left AS lt LEFT JOIN schools_enrollment AS en
        ON lt.id = en.id
    LEFT JOIN schools_grades AS gr
        ON lt.id = gr.id
"""

df = pd.read_sql(sql, engine)

df.head(20)

,id,left_school,enrollment,grades
0,1,Oak Street School,360,K-3
1,2,Roosevelt High School,1001,9-12
2,5,Washington Middle School,450,6-8
3,6,Jefferson High School,927,9-12


In [6]:
# performing math on joining rows

sql = """
    SELECT c2010.geo_name,
           c2010.state_us_abbreviation AS state,
           c2010.p0010001 AS pop_2010,
           c2000.p0010001 AS pop_2000,
           c2010.p0010001 - c2000.p0010001 AS raw_change,
           round( (CAST(c2010.p0010001 AS numeric(8,1)) - c2000.p0010001)
               / c2000.p0010001 * 100, 1 ) AS pct_change
    FROM us_counties_2010 c2010 INNER JOIN us_counties_2000 c2000
    ON c2010.state_fips = c2000.state_fips
       AND c2010.county_fips = c2000.county_fips
       AND c2010.p0010001 <> c2000.p0010001
    ORDER BY pct_change DESC
"""

df = pd.read_sql(sql, engine)

# what we should see is the result of an inner join, which returns only those matching criteria
# state fips, county fips and where population is not the same
# returns only certain tables
# from 2010 table: geo_name, state, pop
# from 2000 table: pop
# raw_change and pct_change are calculated
# with pct_change rounded to 1 decimal

df.head(20)

,geo_name,state,pop_2010,pop_2000,raw_change,pct_change
0,Kendall County,IL,114736,54544,60192,110.4
1,Pinal County,AZ,375770,179727,196043,109.1
2,Flagler County,FL,95696,49832,45864,92.0
3,Lincoln County,SD,44828,24131,20697,85.8
4,Loudoun County,VA,312311,169599,142712,84.1
5,Rockwall County,TX,78337,43080,35257,81.8
6,Forsyth County,GA,175511,98407,77104,78.4
7,Sumter County,FL,93420,53345,40075,75.1
8,Paulding County,GA,142324,81678,60646,74.3
9,Sublette County,WY,10247,5920,4327,73.1


In [10]:
df.shape

(3136, 6)